Using OOD Enablement for the Hugging Face RoBERTA model fine tuned on the Twitter Eval Data Set Use Case

In [ ]:
import os
import sys
import torch
import transformers
import datasets
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tensorflow.keras.utils import get_file
from tempfile import TemporaryDirectory
import numpy as np

ood_path = os.path.abspath('../')
if ood_path not in sys.path:
    sys.path.append(ood_path)

In [ ]:
from ood_enabler.ood_enabler import OODEnabler
from ood_enabler.storage.model_store import ModelStore
from ood_enabler.storage.local_storage import FileSystemStorage
from ood_enabler.util.constants import SavedModelFormat
import datetime

ood_path = os.path.abspath('../')
if ood_path not in sys.path:
    sys.path.append(ood_path)

from ood_enabler.util.constants import SavedModelFormat
from ood_enabler.ood_enabler import OODEnabler
from ood_enabler.storage.model_store import ModelStore
from ood_enabler.storage.local_storage import FileSystemStorage
from ood_enabler.data.pytorch_text_data_handler import PytorchTextDataHandler
from ood_enabler.model_wrapper.pytorch import PytorchWrapper

Create the Twitter Data Handler for In Distribution Normalization 

In [ ]:
dataset = load_dataset("tweet_eval", "sentiment", split='test')
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
archive = None

local_store = None
ds_metadata = {'hf_dataset': ["tweet_eval", "sentiment", "test[:1%]"],  
               'col_name_to_tokenize': "text", 'batch_size': 32, 
               'hf_tokenizer': MODEL }

data_handler = PytorchTextDataHandler()
data_handler.load_dataset(local_store, archive, '.', ds_metadata)

model_store = ModelStore.from_filesystem()

Map:   0%|          | 0/123 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 123
})


Load the Model and create the PyTorch Wrapper

In [ ]:
pretrained_model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model_metadata = {'type': 'pytorch', 'arch': 'roberta', 'ood_thresh_percentile': 20}
model = PytorchWrapper(model_metadata, model=pretrained_model)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model.model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [ ]:
OODEnabler.ood_enable(model, data_handler)

In HF Roberta model condition



Inference progress: 100%|██████████| 4/4 [00:32<00:00,  8.24s/it]

set_threshold for given data handler:Parameter containing:
tensor(1.6271, dtype=torch.float64)
In HF Roberta Condtion for normalization layer



In [ ]:
model.model.eval()

HFSequential(
  (0): HFSequential(
    (0): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0): RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSelfAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): Linear(in_features=768, out_features=768, bias=True)
                  (value): Linear(in_features=768, out_features=768, bias=True)
                  (dropout): Dropout(p=0.1, inplace=False)
                )
                (outpu

Evaluate on the Twitter Eval Data Set

In [ ]:

def preprocess_tweet(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [ ]:
# prep the evaluation dataset 
batch_size = 32
evalset = load_dataset("tweet_eval", "sentiment", split='train')
print(evalset)
tokenizer = AutoTokenizer.from_pretrained(MODEL, model_max_length=512)
for text in evalset["text"]:
    text = preprocess(text)
evalset = evalset.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length') , batched=True)
evalset = evalset.remove_columns(list(filter(lambda x: x not in ["attention_mask", "input_ids"],  evalset.features)))
evalset.set_format(type='torch', columns=['input_ids', 'attention_mask'])
evalloader = torch.utils.data.DataLoader(dataset=evalset, batch_size=batch_size)

Dataset({
    features: ['text', 'label'],
    num_rows: 45615
})


Map:   0%|          | 0/45615 [00:00<?, ? examples/s]

In [ ]:
for i, data in enumerate(evalloader):
    outputs = model.model(data["input_ids"], data["attention_mask"])
    break
print("Printing the final outputs in the example")
print(outputs)

Printing the final outputs in the example
(tensor([[-3.2398,  0.1415,  3.2407],
        [ 0.5372,  1.8618, -2.5646],
        [ 0.0999,  0.4753, -0.7545],
        [-2.3920,  2.2322,  0.1642],
        [-2.8547,  1.7854,  0.8726],
        [-1.8373,  0.9137,  0.8439],
        [-2.7141,  1.4524,  1.2460],
        [ 2.0159, -0.1814, -2.1969],
        [-2.4677,  0.5999,  1.8552],
        [-1.6560,  2.2836, -0.8509],
        [-1.0794,  1.2982, -0.5320],
        [-3.5120,  1.3360,  2.0830],
        [-1.8637, -0.8816,  2.6673],
        [ 2.4629, -0.1700, -2.5207],
        [ 2.0823, -0.1602, -2.4302],
        [-2.1339,  2.6285, -0.8148],
        [-3.1562,  2.1872,  1.0511],
        [-3.5157,  1.9567,  1.2595],
        [ 1.7226,  0.5841, -2.6277],
        [ 0.8891,  0.8631, -2.0804],
        [ 1.5992,  0.7280, -2.5062],
        [-3.7135,  1.0397,  2.5128],
        [-3.4651,  1.7437,  1.9668],
        [-2.6641,  1.0735,  1.2597],
        [-2.2582,  2.6869, -0.4292],
        [-2.0024,  1.6742,  0.36

Save and Load the OOD Enabled Model Out

In [ ]:
# save the OOD enabled model out 
print("Uploading and saving model with torch script format")
path = model_store.upload(model, '.', saved_model_format=SavedModelFormat.TORCHSCRIPT)
print("Path of model: ", path)

Uploading and saving model with torch script format
Saving hf model out with torchscript


/dccstor/edge_cyber_physi/savitha/ood/ood_enabler/model_modifier/pytorch_modifier.py:67: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  temp = torch.tensor(1.0)


Path of model:  ./ood_pytorch_20230821_00-35-43.pt


In [ ]:
# load the OOD enabled model back in 
loaded_model = model_store.load(model_metadata, path, bypass_model_check=True)

/u/savitha/anaconda3/envs/test-ood/lib/python3.9/site-packages/torch/serialization.py:779: UserWarning: 'torch.load' received a zip file that looks like a TorchScript archive dispatching to 'torch.jit.load' (call 'torch.jit.load' directly to silence this warning)
  warnings.warn("'torch.load' received a zip file that looks like a TorchScript archive"


In [ ]:
loaded_model.model

RecursiveScriptModule(
  original_name=HFSequential
  (0): RecursiveScriptModule(
    original_name=HFSequential
    (0): RecursiveScriptModule(
      original_name=RobertaForSequenceClassification
      (roberta): RecursiveScriptModule(
        original_name=RobertaModel
        (embeddings): RecursiveScriptModule(
          original_name=RobertaEmbeddings
          (word_embeddings): RecursiveScriptModule(original_name=Embedding)
          (position_embeddings): RecursiveScriptModule(original_name=Embedding)
          (token_type_embeddings): RecursiveScriptModule(original_name=Embedding)
          (LayerNorm): RecursiveScriptModule(original_name=LayerNorm)
          (dropout): RecursiveScriptModule(original_name=Dropout)
        )
        (encoder): RecursiveScriptModule(
          original_name=RobertaEncoder
          (layer): RecursiveScriptModule(
            original_name=ModuleList
            (0): RecursiveScriptModule(
              original_name=RobertaLayer
              (

In [ ]:
loaded_model.model.eval()

RecursiveScriptModule(
  original_name=HFSequential
  (0): RecursiveScriptModule(
    original_name=HFSequential
    (0): RecursiveScriptModule(
      original_name=RobertaForSequenceClassification
      (roberta): RecursiveScriptModule(
        original_name=RobertaModel
        (embeddings): RecursiveScriptModule(
          original_name=RobertaEmbeddings
          (word_embeddings): RecursiveScriptModule(original_name=Embedding)
          (position_embeddings): RecursiveScriptModule(original_name=Embedding)
          (token_type_embeddings): RecursiveScriptModule(original_name=Embedding)
          (LayerNorm): RecursiveScriptModule(original_name=LayerNorm)
          (dropout): RecursiveScriptModule(original_name=Dropout)
        )
        (encoder): RecursiveScriptModule(
          original_name=RobertaEncoder
          (layer): RecursiveScriptModule(
            original_name=ModuleList
            (0): RecursiveScriptModule(
              original_name=RobertaLayer
              (

In [ ]:
for i, data in enumerate(evalloader):
    # get the inputs; data is a list of [inputs, labels]
    outputs = loaded_model.model(data["input_ids"], data["attention_mask"])
    break
print("Printing the final outputs for the loaded model example")
print(outputs)

Printing the final outputs for the loaded model example
(tensor([[-3.2398,  0.1415,  3.2407],
        [ 0.5372,  1.8618, -2.5646],
        [ 0.0999,  0.4753, -0.7545],
        [-2.3920,  2.2322,  0.1642],
        [-2.8547,  1.7854,  0.8726],
        [-1.8373,  0.9137,  0.8439],
        [-2.7141,  1.4524,  1.2460],
        [ 2.0159, -0.1814, -2.1969],
        [-2.4677,  0.5999,  1.8552],
        [-1.6560,  2.2836, -0.8509],
        [-1.0794,  1.2982, -0.5320],
        [-3.5120,  1.3360,  2.0830],
        [-1.8637, -0.8816,  2.6673],
        [ 2.4629, -0.1700, -2.5207],
        [ 2.0823, -0.1602, -2.4302],
        [-2.1339,  2.6285, -0.8148],
        [-3.1562,  2.1872,  1.0511],
        [-3.5157,  1.9567,  1.2595],
        [ 1.7226,  0.5841, -2.6277],
        [ 0.8891,  0.8631, -2.0804],
        [ 1.5992,  0.7280, -2.5062],
        [-3.7135,  1.0397,  2.5128],
        [-3.4651,  1.7437,  1.9668],
        [-2.6641,  1.0735,  1.2597],
        [-2.2582,  2.6869, -0.4292],
        [-2.0024, 